In [11]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [12]:
df = pd.read_json('../data/all_data_polbert_embeddings.json')
df.head()

,id,text,label,embeddings
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,NaN,"[-0.7777841091000001, -0.2634485662, 0.6810794..."
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,NaN,"[-0.8700618744, -0.4192576408, 0.8261056542, 2..."
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,NaN,"[-1.3931413889, -0.31156659130000003, 0.704142..."
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,NaN,"[-0.7349925041, -0.980086863, 0.6741656065, 0...."
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,NaN,"[-0.6780881882, -0.7579360604000001, 0.4117349..."


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3423 entries, 0 to 3422
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          3423 non-null   object 
 1   text        3423 non-null   object 
 2   label       66 non-null     float64
 3   embeddings  3423 non-null   object 
dtypes: float64(1), object(3)
memory usage: 107.1+ KB


In [14]:
type(df['embeddings'][0])

list

In [15]:
embeddings = np.array(df['embeddings'].tolist())

In [16]:
kmeans_model = KMeans(n_clusters=2, random_state=1)
kmeans_model.fit(embeddings)

/Users/paul/workspace/pejot/supervision-hack-dsc/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=1)

In [17]:
cluster_labels = kmeans_model.labels_

In [18]:
df['cluster'] = cluster_labels

In [19]:
df.head()

,id,text,label,embeddings,cluster
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,NaN,"[-0.7777841091000001, -0.2634485662, 0.6810794...",1
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,NaN,"[-0.8700618744, -0.4192576408, 0.8261056542, 2...",1
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,NaN,"[-1.3931413889, -0.31156659130000003, 0.704142...",0
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,NaN,"[-0.7349925041, -0.980086863, 0.6741656065, 0....",0
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,NaN,"[-0.6780881882, -0.7579360604000001, 0.4117349...",1


In [20]:
distribution_positives = df[df['label'] == 1]['cluster'].value_counts()
distribution_positives

cluster
0    24
1     3
Name: count, dtype: int64

In [21]:
print('Cluster with most positives:', distribution_positives.idxmax())

Cluster with most positives: 0


In [22]:
distribution_negatives = df[df['label'] == 0]['cluster'].value_counts()
distribution_negatives

cluster
1    20
0    19
Name: count, dtype: int64

In [30]:
print(f'''Suspected cluster of negatives: 1
Correctly classified positives: {distribution_positives.loc[0]} out of {distribution_positives.sum()}
Correctly classified negatives: {distribution_negatives.loc[1]} out of {distribution_negatives.sum()}
''')

Suspected cluster of negatives: 1
Correctly classified positives: 24 out of 27
Correctly classified negatives: 20 out of 39



In [33]:
print(f'''Confusion matrix:
{df.groupby(['label', 'cluster']).size().unstack('cluster')}
''')

Confusion matrix:
cluster   0   1
label          
0.0      19  20
1.0      24   3



In [36]:
print(f'''Metrics:
Accuracy: {(distribution_positives.loc[0] + distribution_negatives.loc[1]) / (distribution_positives.sum() + distribution_negatives.sum())}
Precision: {distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_negatives.loc[0])}
Recall: {distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_positives.loc[1])}
Support: {distribution_positives.loc[0] + distribution_positives.loc[1]}
F1: {2 * (distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_negatives.loc[0])) * (distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_positives.loc[1])) / ((distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_negatives.loc[0])) + (distribution_positives.loc[0] / (distribution_positives.loc[0] + distribution_positives.loc[1])))}
''')

Metrics:
Accuracy: 0.6666666666666666
Precision: 0.5581395348837209
Recall: 0.8888888888888888
Support: 27
F1: 0.6857142857142857



In [37]:
df_with_labels = df[(df['label'] == 1) | (df['label'] == 0)]

In [38]:
df_with_labels.shape

(66, 5)

In [39]:
print('Cluster 1 (suspected negatives):')

Cluster 1 (suspected negatives):


In [40]:
df_with_labels[df_with_labels['cluster'] == 1]['text']

3372     konto bankowe anonimowe pewne na 100 w jednym...
3373    dostępne jest konto bankowe o wysokim poziomie...
3374    dostępne jest konto bankowe które gwarantuje p...
3375    posiadamy konto bankowe które zapewnia absolut...
3376    mamy dostępne konto bankowe które zapewnia peł...
3377    sprzedam konta bankowe bez obciążeń dostępne c...
3378    oferuję do sprzedaży konta bankowe bez żadnych...
3379    mam na sprzedaż konta bankowe które są wolne o...
3380    posiadam na sprzedaż konta bankowe które są ca...
3381    mam na sprzedaż konta bankowe które są wolne o...
3385    witam sprzedajemy konta bankowe na słupa bez k...
3386                                                     
3389    zatrudnimy doracę doradczynię ds kontaktu z kl...
3390    jeśli w pracy nie lubisz rutyny szukasz stałeg...
3414    zatrudnimy osobę do przeprowadzania badań tele...
3415    zatrudnimy kierowcę do przewozu towarów niebez...
3416    poszukujemy informatyka z umiejętnościami włam...
3417    firma 

In [126]:
### This was used to generate new validation data using GPT-4

# df_correctly_clustered_positives = df_with_labels[(df_with_labels['cluster'] == 0) & (df_with_labels['label'] == 1)]
# df_correctly_clustered_negatives = df_with_labels[(df_with_labels['cluster'] == 1) & (df_with_labels['label'] == 0)]
#
# df_correctly_clustered = pd.concat([df_correctly_clustered_positives, df_correctly_clustered_negatives])
# df_correctly_clustered = df_correctly_clustered[['text', 'label', 'cluster']]
# df_correctly_clustered.head()
#
# df_correctly_clustered.to_csv('../data/correctly_clustered.csv', index=False)

In [41]:
df.head()

,id,text,label,embeddings,cluster
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,NaN,"[-0.7777841091000001, -0.2634485662, 0.6810794...",1
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,NaN,"[-0.8700618744, -0.4192576408, 0.8261056542, 2...",1
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,NaN,"[-1.3931413889, -0.31156659130000003, 0.704142...",0
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,NaN,"[-0.7349925041, -0.980086863, 0.6741656065, 0....",0
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,NaN,"[-0.6780881882, -0.7579360604000001, 0.4117349...",1


In [42]:
# cluster 0 are the positives
# cluster 1 are the negatives
df['label'] = df['cluster'].apply(lambda x: 1 if x == 0 else 0)
df.drop(columns=['cluster'], inplace=True)

In [43]:
df.head()

,id,text,label,embeddings
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,0,"[-0.7777841091000001, -0.2634485662, 0.6810794..."
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,0,"[-0.8700618744, -0.4192576408, 0.8261056542, 2..."
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,1,"[-1.3931413889, -0.31156659130000003, 0.704142..."
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,1,"[-0.7349925041, -0.980086863, 0.6741656065, 0...."
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,0,"[-0.6780881882, -0.7579360604000001, 0.4117349..."


In [44]:
df.to_csv('../data/labeled_data_for_supervised.csv', index=False)
with open('../data/labeled_data_for_supervised.json', 'w', encoding='utf-8') as f:
  df.to_json(f, force_ascii=False, orient='records')